# Scrabble-Playing Robot

## Imports

In [ ]:
import logging
from copy import copy
from enum import Enum
import matplotlib.pyplot as plt
import pydot
from IPython.display import HTML, SVG, display, Image

import numpy as np
from pydrake.all import (
    AbstractValue,
    Demultiplexer,
    DiagramBuilder,
    InputPortIndex,
    JointIndex,
    LeafSystem,
    Parser,
    PiecewisePolynomial,
    PiecewisePose,
    PointCloud,
    PortSwitch,
    RigidTransform,
    Simulator,
    StateInterpolatorWithDiscreteDerivative,
    StartMeshcat,
    UniformlyRandomRotationMatrix,
    QuaternionFloatingJoint,
    Role,
    AngleAxis,
    RigidTransform,
    ModelVisualizer,
    MeshcatVisualizer
)
from pydrake.math import (
    RotationMatrix
)

from manipulation import running_as_notebook
from manipulation.meshcat_utils import AddMeshcatTriad
from manipulation.pick import MakeGripperPoseTrajectory
from manipulation.scenarios import AddIiwaDifferentialIK, ycb
from manipulation.station import (
    MakeHardwareStation,
    load_scenario,
)
import importlib


class NoDiffIKWarnings(logging.Filter):
    def filter(self, record):
        return not record.getMessage().startswith("Differential IK")


logging.getLogger("drake").addFilter(NoDiffIKWarnings())

In [ ]:
import constants
from constants import *
importlib.reload(constants)

import letter_tree
import board
import solver
import scorer
import holder
import bag

importlib.reload(letter_tree)
importlib.reload(board)
importlib.reload(solver)
importlib.reload(scorer)
importlib.reload(holder)
importlib.reload(bag)

from board import Board
from solver import SolveState
from holder import Holder
from letter_tree import basic_english
from bag import *

In [ ]:
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at https://4b17a40c-416d-4160-9e25-b767157b933d.deepnoteproject.com/7000/


## Game Constants

In [ ]:
BOARD_SIZE = 5
PIECES_IN_HOLDER = 3
WORD_DICTIONARY = basic_english()
z = 0.0026 # height of tile
delta = 0.02 # distance between adjacent board positions

# location to teleport preloaded pieces from - just a placeholder location
bag_loc = [0.2, 0.2, 0]

# location to pickup randomly generated tile when refilling the holder
generation_loc = [-0.2, -0.2, 0.0]

tile_to_count = tile_to_count_tiny
# tile_to_count = tile_to_count_small

In [ ]:
def fill_bag():
    pos_to_id = dict()
    return Bag(tile_to_count)

def get_tile_body_name(tile):
    return f"{tile}1"

def get_tile_instance_name(tile, tile_id):
    return f"{tile}{tile_id}"

def get_tile_joint_name(tile, tile_id):
    return f"{tile}{tile_id}_joint"

## Transformations

In [ ]:
def board_to_world(board_state, pos, zi=0.0):
    """
    Gets the pose of a board piece with respect to the world.

    Parameters:
    - board_state (Board): The Board
    - pos (tuple): The (row, col) coordinates of the piece in the internal board state
    - zi (int): The height of the piece

    Returns:
    - RigidBodyTransform: The transform of the board piece with respect to the world.
    """
    TAG = "board_to_world"
    shifted = (np.array(board_state.get_center_pos()) - np.array(pos)) * delta
    X_WP = RigidTransform(RotationMatrix(), [shifted[0], shifted[1], zi])
    # print(f"{TAG}: pos: {pos} and shifted: {shifted}")
    return X_WP


def holder_to_world(idx, zi=0.0):
    """
    Gets the pose of the holder piece with respect to the world.

    Parameters:
    - idx (int): The position of the piece in the internal holder state
    - zi (int): The height of the piece

    Returns:
    - RigidBodyTransform: The transform of the holder piece with respect to the world.
    """
    TAG = "holder_to_world"
    x = -10 * delta
    y = idx * delta
    X_WP = RigidTransform(RotationMatrix(), [x, y, zi])
    # print(f"{TAG} pos: ({x},{y}) and idx: {idx}")
    return X_WP


def bag_to_world(i, j, zi=0.0):
    """
    Gets the pose of a bag piece with respect to the world.

    Parameters:
    - i (int): The row coordinate of the piece in the bag
    - j (int): The col coordinate of the piece in the bag
    - zi (int): The height of the piece

    Returns:
    - RigidBodyTransform: The transform of the bag piece with respect to the world.
    """
    x = -0.25 + delta * j
    y = -0.2 - delta * i
    X_WP = RigidTransform(RotationMatrix(), [x, y, zi])
    return X_WP


def Get_X_SP(X_WS, X_WP):
    """
    Gets the pose of a piece with respect to the suction tip link.

    Parameters:
    - X_WS (RigidBodyTransform): The transform of the tip link with respect to the world
    - X_WP (RigidBodyTransform): The transform of the piece with respect to the world

    Returns:
    - RigidBodyTransform: The transform of the piece with respect to the tip link.
    """
    return RigidTransform.multiply(X_WS.inverse(), X_WP)

## Create Scene with Initial Board and Holder States

In [ ]:
def create_scene():
    '''
    Writes the scenario data for the initial scene of the board. This includes:
    1. iiwa robot with a camera welded to the seventh joint
    2. Table to play the game
    3. Scrabble Board
    4. Holder for the the letters in play
    
    Additionally, all the letters are loaded in with the locations defined by `locations`

    Returns the scenario data (text) for the Scrabble game
    '''
    scenario_data = f"""
directives: 
- add_model:
    name: iiwa
    file: package://drake/manipulation/models/iiwa_description/iiwa7/iiwa7_no_collision.sdf
    default_joint_positions:
        iiwa_joint_1: [0]       # Base joint - rotate the base to face the board
        iiwa_joint_2: [0.1]     # Slight lift of the second joint to lower the end effector
        iiwa_joint_3: [0]       # Keep the third joint straight for now
        iiwa_joint_4: [-1.2]     # Slight bend of the fourth joint to bring the arm closer to the board
        iiwa_joint_5: [0]       # Keep the fifth joint straight for now
        iiwa_joint_6: [1.6]    # Bend the sixth joint to position the end effector downwards
        iiwa_joint_7: [0] 
- add_weld:
    parent: world
    child: iiwa::iiwa_link_0
    X_PC:
        translation: [-0.6, 0, 0]
- add_model:
    name: suction_base
    file: package://manipulation/suction/suction-base.urdf
- add_weld:
    parent: iiwa::iiwa_link_7
    child: suction_base::baseLink
    X_PC:
        translation: [0, 0, 0.01]
        rotation: !Rpy {{ deg: [0, 0, 0] }}
- add_model:
    name: suction_head
    file: file:///work/models/suction_head.urdf
- add_weld:
    parent: iiwa::iiwa_link_7
    child: suction_head::headLink
    X_PC:
        translation: [0, 0, 0.10]
        rotation: !Rpy {{ deg: [0, 0, 0] }}
- add_model:
    name: work_table
    file: file:///work/models/extra_heavy_duty_table_surface_only_collision.sdf
- add_weld:
    parent: world
    child: work_table::link
    X_PC:
        translation: [0, 0, -0.7645]
- add_model:
    name: board
    file: file:///work/models/scrabble_board.sdf
    default_free_body_pose:
        scrabble_board: # Change here!
            translation: [0,0,0]
            rotation: !Rpy {{ deg: [0, 0, 0] }} 
- add_weld:
    parent: world
    child: scrabble_board
    X_PC:
        translation: [0, 0, 0]
        rotation: !Rpy {{ deg: [0, 0, 0] }}
"""

    for tile, tile_id in bag_state.get_all_tiles():
        scenario_data += f"""
- add_model:
    name: {tile}{str(tile_id)}
    file: file:///work/models/{tile}1.sdf
    default_free_body_pose:
        {tile}1: # Change here!
            translation: {bag_loc}
            rotation: !Rpy {{ deg: [0, 0, 0] }}  
"""

    scenario_data += f"""
model_drivers:
    iiwa+suction_base+suction_head: !InverseDynamicsDriver {{}}
"""
    return scenario_data

In [ ]:
def teleport_piece(X, tile, tile_id, plant, plant_context):
    """
    Teleport tile to a given pose.
    """
    TAG = "teleport_piece"
    body_name = get_tile_body_name(tile)
    instance_name = get_tile_instance_name(tile, tile_id)
    plant.SetFreeBodyPose(
        plant_context,
        plant.GetBodyByName(body_name, plant.GetModelInstanceByName(instance_name)),
        X)
    # print(f"{TAG}: Teleporting tile '{instance_name}' to {X_WP.translation()}.")

In [ ]:
def set_board_state(board_state: Board, station, context, plant, plant_context, X_WS):
    """
    Used at the beginning of the game to set the board.
    Given the current board, place the scrabble pieces on the board.
    """
    TAG = "set_board_state"

    for pos in board_state.all_positions():

        if board_state.is_empty(pos):
            continue

        tile = board_state.get_tile(pos)
        tile_id = pos_to_id[pos]

        # transform of the piece with respect to the world
        transform = board_to_world(board_state, pos, z)
        X_WP = RigidTransform(transform.rotation().MakeZRotation(-np.pi/2), transform.translation())
        X_SP = Get_X_SP(X_WS, X_WP)
        joint_name = get_tile_joint_name(tile, tile_id)
        instance_name = get_tile_instance_name(tile, tile_id)
        model_joint = plant.GetJointByName(joint_name)
        model_joint.SetDefaultPose(RigidTransform([0, 0, 0.1]))
        model_joint.Unlock(plant_context)
        teleport_piece(X_SP, tile, tile_id, plant, plant_context)
        # print(f"{TAG}: Set pose of {instance_name} to {X_SP.translation()}.")

In [ ]:
def set_holder_state(holder_state, station, context, plant, plant_context, X_WS):
    """
    Used at the beginning of the game to set up the holder.
    """
    TAG = "set_holder_state"

    for i in holder_state.get_filled_positions():
        transform = holder_to_world(i, z)
        X_WP = RigidTransform(transform.rotation().MakeZRotation(-np.pi/2), transform.translation())
        X_SP = Get_X_SP(X_WS, X_WP)
        tile, tile_id = holder_state.get_tile(i)
        joint_name = get_tile_joint_name(tile, tile_id)
        instance_name = get_tile_instance_name(tile, tile_id)
        model_joint = plant.GetJointByName(joint_name)
        model_joint.SetDefaultPose(RigidTransform([0, 0, 0.1]))
        model_joint.Unlock(plant_context)
        teleport_piece(X_SP, tile, tile_id, plant, plant_context)
        # print(f"{TAG}: Set pose of {instance_name} to {X_SP.translation()}.")

In [ ]:
def set_bag_state(bag_state, station, context, plant, plant_context, X_WS):
    """
    Teleport bag pieces to correct locations
    """
    TAG = "set_bag_state"
    l = bag_state.get_all_tiles()

    for tile, tile_id in bag_state.get_all_tiles():

        bag_pos = bag_state.tile_to_pos[(tile, tile_id)]
        i, j = bag_pos
        transform = bag_to_world(i, j, z)
        X_WP = RigidTransform(transform.rotation().MakeZRotation(-np.pi/2), transform.translation())
        X_SP = Get_X_SP(X_WS, X_WP)
        joint_name = get_tile_joint_name(tile, tile_id)
        instance_name = get_tile_instance_name(tile, tile_id)
        model_joint = plant.GetJointByName(joint_name)
        model_joint.SetDefaultPose(RigidTransform([0, 0, 0.1]))
        model_joint.Unlock(plant_context)
        teleport_piece(X_SP, tile, tile_id, plant, plant_context)
        # print(f"{TAG}: Set pose of {instance_name} to {X_SP.translation()}.")

## PlannerState Enum

In [ ]:
class PlannerState(Enum):
    """
    Represents different states of the scrabble game.
    """
    CREATE_WORD = 1
    PLAN_MOVE_TO_PICKUP_IN_HOLDER = 2
    MOVE_TO_PICKUP_IN_HOLDER = 3
    PICKUP_IN_HOLDER = 4
    MOVE_TO_DROP_OFF_ON_BOARD = 5
    DROP_OFF_ON_BOARD = 6

    GENERATE_PIECE = 7
    MOVE_TO_PICKUP_IN_GENERATION = 8
    PICKUP_IN_GENERATION = 9
    MOVE_TO_DROP_OFF_IN_HOLDER = 10
    DROP_OFF_IN_HOLDER = 11

    END_GAME = 12

## Initialize Game Variables

In [ ]:
rng = np.random.default_rng(135)  # this is for python

pos_to_id = dict() # maps positions of tiles on board to their ids
bag_state: Bag = fill_bag()

In [ ]:
def MakeGripperFrames(X_G, t0=0):
    """
    Takes a partial specification with X_G["initial"], X_G["pick"], and
    X_G["place"], and returns a X_G and times with all of the pick and place
    frames populated.
    """
    # pregrasp is negative y in the gripper frame (see the figure!).
    X_GgraspGpregrasp = RigidTransform([0, 0, -0.2])

    X_G["prepick"] = X_G["pick"] @ X_GgraspGpregrasp
    X_G["preplace"] = X_G["place"] @ X_GgraspGpregrasp

    # I'll interpolate a halfway orientation by converting to axis angle and
    # halving the angle.
    X_GinitialGprepick = X_G["initial"].inverse() @ X_G["prepick"]
    angle_axis = X_GinitialGprepick.rotation().ToAngleAxis()
    X_GinitialGprepare = RigidTransform(
        AngleAxis(angle=angle_axis.angle() / 2.0, axis=angle_axis.axis()),
        X_GinitialGprepick.translation() / 2.0,
    )
    X_G["prepare"] = X_G["initial"] @ X_GinitialGprepare
    p_G = np.array(X_G["prepare"].translation())
    p_G[2] = 0.5
    # To avoid hitting the cameras, make sure the point satisfies x - y < .5
    if p_G[0] - p_G[1] < 0.5:
        scale = 0.5 / (p_G[0] - p_G[1])
        p_G[:1] /= scale
    X_G["prepare"].set_translation(p_G)

    X_GprepickGpreplace = X_G["prepick"].inverse() @ X_G["preplace"]
    angle_axis = X_GprepickGpreplace.rotation().ToAngleAxis()
    X_GprepickGclearance = RigidTransform(
        AngleAxis(angle=angle_axis.angle() / 2.0, axis=angle_axis.axis()),
        X_GprepickGpreplace.translation() / 2.0,
    )
    X_G["clearance"] = X_G["prepick"] @ X_GprepickGclearance
    p_G = np.array(X_G["clearance"].translation())
    p_G[2] = 0.5
    # To avoid hitting the cameras, make sure the point satisfies x - y < .5
    if p_G[0] - p_G[1] < 0.5:
        scale = 0.5 / (p_G[0] - p_G[1])
        p_G[:1] /= scale
    X_G["clearance"].set_translation(p_G)

    # Now let's set the timing
    times = {"initial": t0}
    prepare_time = 10.0 * np.linalg.norm(X_GinitialGprepare.translation())
    times["prepare"] = times["initial"] + prepare_time
    times["prepick"] = times["prepare"] + prepare_time
    # Allow some time for the gripper to close.
    times["pick_start"] = times["prepick"] + 2.0
    times["pick_end"] = times["pick_start"] + 2.0
    X_G["pick_start"] = X_G["pick"]
    X_G["pick_end"] = X_G["pick"]
    times["postpick"] = times["pick_end"] + 2.0
    X_G["postpick"] = X_G["prepick"]
    time_to_from_clearance = 10.0 * np.linalg.norm(
        X_GprepickGclearance.translation()
    )
    times["clearance"] = times["postpick"] + time_to_from_clearance
    times["preplace"] = times["clearance"] + time_to_from_clearance
    times["place_start"] = times["preplace"] + 2.0
    times["place_end"] = times["place_start"] + 2.0
    X_G["place_start"] = X_G["place"]
    X_G["place_end"] = X_G["place"]
    times["postplace"] = times["place_end"] + 2.0
    X_G["postplace"] = X_G["preplace"]

    return X_G, times


## Planner Class

In [ ]:
class Planner(LeafSystem):

    def __init__(self, plant, bag_state: Bag, board_state: Board, holder_state: Holder):
        LeafSystem.__init__(self)

        self.bag_state = bag_state
        self.board_state = board_state
        self.holder_state = holder_state
        self.overall_score = 0
        self.word = ''
        self.letters_to_add = []
        self.word_score = 0

        self.position_in_holder = 0
        self.drop_off_pos_board = None
        self.drop_off_pos_holder = None

        self.attached_tile = None
        self.tile_to_attach = None
        
        self._gripper_body_index = plant.GetBodyByName("tipLink").index()
        self.DeclareAbstractInputPort(
            "body_poses", AbstractValue.Make([RigidTransform()])
        )

        self._mode_index = self.DeclareAbstractState(
            AbstractValue.Make(PlannerState.CREATE_WORD)
        )
        self._traj_X_G_index = self.DeclareAbstractState(
            AbstractValue.Make(PiecewisePose())
        )
        self._times_index = self.DeclareAbstractState(
            AbstractValue.Make({"initial": 0.0})
        )
        self._attempts_index = self.DeclareDiscreteState(1)

        self.DeclareAbstractOutputPort(
            "X_WG",
            lambda: AbstractValue.Make(RigidTransform()),
            self.CalcGripperPose,
        )
        port = self.DeclareVectorOutputPort("suction_on", 1, self.DoCalcOutput)
        port.disable_caching_by_default()

        # For GoHome mode.
        num_positions = 7
        self._iiwa_position_index = self.DeclareVectorInputPort(
            "iiwa_position", num_positions
        ).get_index()
        self.DeclareAbstractOutputPort(
            "control_mode",
            lambda: AbstractValue.Make(InputPortIndex(0)),
            self.CalcControlMode,
        )
        self.DeclareAbstractOutputPort(
            "reset_diff_ik",
            lambda: AbstractValue.Make(False),
            self.CalcDiffIKReset,
        )
        self._q0_index = self.DeclareDiscreteState(num_positions)  # for q0
        self._traj_q_index = self.DeclareAbstractState(
            AbstractValue.Make(PiecewisePolynomial())
        )
        self.DeclareVectorOutputPort(
            "iiwa_position_command", num_positions, self.CalcIiwaPosition
        )
        self.DeclareInitializationDiscreteUpdateEvent(self.Initialize)

        self.DeclarePeriodicUnrestrictedUpdateEvent(0.1, 0.0, self.Update)


    def GamePlay(self):
        """
        Call Scrabble board algorithm with the holder pieces and board state to find valid words, 
        and from those words find the highest scoring word.

        Returns:
        - tuple: The word to play (str), letters and positions (list), and the score of the word (int).
        """
        TAG = "game_play"
        solver = SolveState(WORD_DICTIONARY, self.board_state, self.holder_state.get_just_tiles())
        highest_score_word = scorer.get_highest_score_word(solver)

        if highest_score_word is not None:
            word, positions, word_score = highest_score_word
            print(f"{TAG}: Found word '{word}' with positions {positions} and score {word_score}")
        else:
            print(f"{TAG}: TODO") # could not find a word so need to replace rack
            return False

        letters_to_add = [(letter, pos) for letter, pos in zip(word, positions) if self.board_state.is_empty(pos)]
        return word, letters_to_add, word_score


    def MakeTrajectory(self, X_G, context, state):
        """
        Composes a trajectory for the iiwa given a dictionary of RigidBodyTransforms.
        """
        X_G, times = MakeGripperFrames(X_G, t0=context.get_time())
        state.get_mutable_abstract_state(int(self._times_index)).set_value(
            times
        )
        if False:  # Useful for debugging
            for key in X_G:
                AddMeshcatTriad(meshcat, key, X_PT=X_G[key])
        traj_X_G = MakeGripperPoseTrajectory(X_G, times)
        state.get_mutable_abstract_state(int(self._traj_X_G_index)).set_value(traj_X_G)


    def Update(self, context, state):
        """
        Periodically called update function that runs the game loop.
        """

        mode = context.get_abstract_state(int(self._mode_index)).get_value()
        current_time = context.get_time()
        times = context.get_abstract_state(int(self._times_index)).get_value()
        suction_space = 0
        suction_pickup = [0,0,0.023]
        suction_dropoff = [0,0,0.024]
        R = RotationMatrix().MakeYRotation(np.pi)

        print(f"mode: {mode}")
        print(f"current time: {current_time}")
        print(f"holder: {self.holder_state}")
        print(f"board: \n{self.board_state}")
        print(f"times: {times}")

        # find best word with scrabble solver
        if mode == PlannerState.CREATE_WORD:
            result = self.GamePlay()

            if not result:
                mode = PlannerState.END_GAME
            else:
                self.word, self.letters_to_add, self.word_score = result
                mode = PlannerState.PLAN_MOVE_TO_PICKUP_IN_HOLDER
        
        elif mode == PlannerState.PLAN_MOVE_TO_PICKUP_IN_HOLDER:
            tile, board_position = self.letters_to_add.pop()
            self.loc_in_holder, tile_id = self.holder_state.get_tile_position(tile)
            self.drop_off_pos_board = board_position
        
            initial_pose = self.get_input_port(0).Eval(context)[
                int(self._gripper_body_index)
            ]
            
            X_G = {
                "initial": initial_pose, 
                "pick": RigidTransform(R, holder_to_world(self.loc_in_holder, suction_space).translation() + suction_pickup),
                "place": RigidTransform(R, board_to_world(self.board_state, self.drop_off_pos_board, suction_space).translation() + suction_dropoff)
            }
            self.MakeTrajectory(X_G, context, state)
            self.tile_to_attach = (tile, tile_id)
            mode = PlannerState.MOVE_TO_PICKUP_IN_HOLDER

        # moves to pickup location in holder
        elif mode == PlannerState.MOVE_TO_PICKUP_IN_HOLDER:
            if context.get_time() > times["pick_start"]:
                mode = PlannerState.PICKUP_IN_HOLDER
        
        # picks up piece and moves to drop off on board
        elif mode == PlannerState.PICKUP_IN_HOLDER:
            self.attached_tile = self.tile_to_attach
            self.tile_to_attach = None
            self.holder_state.delete_tile(self.loc_in_holder)
            mode = PlannerState.MOVE_TO_DROP_OFF_ON_BOARD
        
        # picks up piece and moves to drop off on board
        elif mode == PlannerState.MOVE_TO_DROP_OFF_ON_BOARD:
            if context.get_time() > times["place_start"]:
                mode = PlannerState.DROP_OFF_ON_BOARD

        # drops piece, moves from board drop off to neutral position
        elif mode == PlannerState.DROP_OFF_ON_BOARD:
            if context.get_time() > times["postplace"]:
                if len(self.letters_to_add) == 0 and not self.bag_state.is_empty():
                    mode = PlannerState.GENERATE_PIECE
                    self.overall_score += self.word_score
                elif len(self.letters_to_add) > 0:
                    mode = PlannerState.PLAN_MOVE_TO_PICKUP_IN_HOLDER
                else: 
                    mode = PlannerState.END_GAME
            self.board_state.set_tile(self.drop_off_pos_board, self.attached_tile[0]) # added
        
        # generates piece and teleports to generation location
        elif mode == PlannerState.GENERATE_PIECE:
            
            tile, tile_id = self.bag_state.draw_tile()
            body_name = get_tile_body_name(tile)
            instance_name = get_tile_instance_name(tile, tile_id)
            self.tile_to_attach = (tile, tile_id)

            initial_pose = self.get_input_port(0).Eval(context)[
                int(self._gripper_body_index)
            ]

            pos_i, pos_j = self.bag_state.tile_to_pos[(tile, tile_id)]
            generation_loc = bag_to_world(pos_i, pos_j, suction_space)

            empty_positions = self.holder_state.get_empty_positions()
            pos_idx = empty_positions[0]
            self.drop_off_pos_holder = pos_idx
            
            X_G = {
                "initial": initial_pose, 
                "pick": RigidTransform(R, generation_loc.translation() + suction_pickup),
                "place": RigidTransform(R, holder_to_world(pos_idx, suction_space).translation() + suction_dropoff)
            }

            self.MakeTrajectory(X_G, context, state)
            mode = PlannerState.MOVE_TO_PICKUP_IN_GENERATION

        # picks up piece, and moves to dropoff in holder
        elif mode == PlannerState.MOVE_TO_PICKUP_IN_GENERATION:
            if context.get_time() >= times["pick_start"]:
                mode = PlannerState.PICKUP_IN_GENERATION
        
        elif mode == PlannerState.PICKUP_IN_GENERATION:
            self.attached_tile = self.tile_to_attach
            self.tile_to_attach = None
            mode = PlannerState.MOVE_TO_DROP_OFF_IN_HOLDER

        elif mode == PlannerState.MOVE_TO_DROP_OFF_IN_HOLDER:
            if context.get_time() >= times["place_start"]:
                mode = PlannerState.DROP_OFF_IN_HOLDER

        # drops piece, moves from dropoff in holder to neutral position
        elif mode == PlannerState.DROP_OFF_IN_HOLDER:
            if context.get_time() > times["postplace"]:
                tile, tile_id = self.attached_tile
                self.holder_state.set_tile(tile, tile_id, self.drop_off_pos_holder)
                self.attached_tile = None
                if self.holder_state.is_full():
                    mode = PlannerState.CREATE_WORD
                elif not self.holder_state.is_full() and not self.bag_state.is_empty():
                    mode = PlannerState.GENERATE_PIECE
                else: 
                    print("No more plays left to play")
                    mode = PlannerState.END_GAME
        else:
            print(f"Game over! Your final score is {self.overall_score}. Please press `esc` to stop the simulation.")
        
        state.get_mutable_abstract_state(int(self._mode_index)).set_value(mode)


    def start_time(self, context):
        return (
            context.get_abstract_state(int(self._traj_X_G_index))
            .get_value()
            .start_time()
        )


    def end_time(self, context):
        return (
            context.get_abstract_state(int(self._traj_X_G_index))
            .get_value()
            .end_time()
        )


    def CalcGripperPose(self, context, output):
        context.get_abstract_state(int(self._mode_index)).get_value()

        traj_X_G = context.get_abstract_state(
            int(self._traj_X_G_index)
        ).get_value()
        if traj_X_G.get_number_of_segments() > 0 and traj_X_G.is_time_in_range(
            context.get_time()
        ):
            # Evaluate the trajectory at the current time, and write it to the
            # output port.
            output.set_value(
                context.get_abstract_state(int(self._traj_X_G_index))
                .get_value()
                .GetPose(context.get_time())
            )
            return
        
        # Command the current position (note: this is not particularly good if the velocity is non-zero)
        output.set_value(
            self.get_input_port(0).Eval(context)[int(self._gripper_body_index)]
        )


    def DoCalcOutput(self, context, output):
        mode = context.get_abstract_state(int(self._mode_index)).get_value()

        if mode == PlannerState.PICKUP_IN_HOLDER \
            or mode == PlannerState.MOVE_TO_DROP_OFF_ON_BOARD \
            or mode == PlannerState.PICKUP_IN_GENERATION \
            or mode == PlannerState.MOVE_TO_DROP_OFF_IN_HOLDER:
            output.SetAtIndex(0, 1)

        else:
            output.SetAtIndex(0, 0)


    def CalcControlMode(self, context, output):
        mode = context.get_abstract_state(int(self._mode_index)).get_value()

        output.set_value(InputPortIndex(1))  # Diff IK


    def CalcDiffIKReset(self, context, output):
        mode = context.get_abstract_state(int(self._mode_index)).get_value()

        output.set_value(False)


    def Initialize(self, context, discrete_state):
        discrete_state.set_value(
            int(self._q0_index),
            self.get_input_port(int(self._iiwa_position_index)).Eval(context),
        )


    def CalcIiwaPosition(self, context, output):
        traj_q = context.get_mutable_abstract_state(
            int(self._traj_q_index)
        ).get_value()

        output.SetFromVector(traj_q.value(context.get_time()))

## Run game

In [ ]:
def set_up_sample_game(bag_state):
    """
    Sets up a sample game of:
    - A 7x7 game board with 'a' initialized in the center
    - A holder of size 3 initialized with the letters 't' and 'h'
    - A bag with the letters 'n' and 'o'
    """

    # initialize empty board
    board_size = 7
    board_state = Board(board_size)

    # add start tiles to board
    board_tiles = ["a"]
    board_positions = [(3, 3)]

    for start_tile, start_pos in zip(board_tiles, board_positions):
        board_state.set_tile(start_pos, start_tile)
        idxb = bag_state.remove_tile(start_tile)
        pos_to_id[start_pos] = idxb

    # add holder tiles
    holder_tiles = ["t", "h"]
    assert len(holder_tiles) <= PIECES_IN_HOLDER

    holder_state = Holder(PIECES_IN_HOLDER)
    for i, tile in enumerate(holder_tiles):
        tile_id = bag_state.remove_tile(tile)
        holder_state.set_tile(tile, tile_id, i)

    return bag_state, board_state, holder_state

In [ ]:
def set_up_sample_game2(bag_state):

    # initialize empty board
    board_size = 7
    board_state = Board(board_size)

    # add start tiles to board
    board_tiles = ["r"]
    board_positions = [(3, 3)]

    for start_tile, start_pos in zip(board_tiles, board_positions):
        board_state.set_tile(start_pos, start_tile)
        idxb = bag_state.remove_tile(start_tile)
        pos_to_id[start_pos] = idxb

    # add holder tiles
    holder_tiles = ["w", "o", "d", "s", "i"]

    holder_state = Holder(5)
    for i, tile in enumerate(holder_tiles):
        tile_id = bag_state.remove_tile(tile)
        holder_state.set_tile(tile, tile_id, i)

    return bag_state, board_state, holder_state

In [ ]:
def add_suction_joints(parser):
    plant = parser.plant()
    suction_body = plant.GetBodyByName("tipLink")
    constraints = {}
    for tile, tile_id in bag_state.get_all_tiles():
        body_name = get_tile_body_name(tile)
        instance_name = get_tile_instance_name(tile, tile_id)

        model_instance_id = plant.GetModelInstanceByName(instance_name)
        model_body = plant.GetBodyByName(body_name, model_instance_id)
        model_joint = plant.AddJoint(
            QuaternionFloatingJoint(
                get_tile_joint_name(tile, tile_id),
                suction_body.body_frame(),
                model_body.body_frame(),
            )
        )
        X_SuctionTile = RigidTransform([0, 0, 0.1])
        model_joint.SetDefaultPose(X_SuctionTile)

In [ ]:
def run_game():

    bag_state = fill_bag()
    meshcat.Delete()
    builder = DiagramBuilder()

    scenario_data = create_scene()
    scenario = load_scenario(data=scenario_data)

    station = builder.AddSystem(
        MakeHardwareStation(
            scenario, meshcat, parser_prefinalize_callback=add_suction_joints
        )
    )

    bag_state, board_state, holder_state = set_up_sample_game(bag_state)
    plant = station.GetSubsystemByName("plant")
    planner = builder.AddSystem(Planner(plant, bag_state, board_state, holder_state))

    builder.Connect(
        station.GetOutputPort("body_poses"),
        planner.GetInputPort("body_poses"),
    )
    demux = builder.AddSystem(Demultiplexer([7, 7]))
    builder.Connect(station.GetOutputPort("iiwa+suction_base+suction_head.state_estimated"), demux.get_input_port())
    builder.Connect(
        demux.get_output_port(0),
        planner.GetInputPort("iiwa_position"),
    )

    robot = station.GetSubsystemByName(
        "iiwa+suction_base+suction_head.controller"
    ).get_multibody_plant_for_control()

    desired_state_from_position = builder.AddSystem(
        StateInterpolatorWithDiscreteDerivative(
            7,
            plant.time_step(),
            suppress_initial_transient=True,
        )
    )
    desired_state_from_position.set_name(
        "iiwa.desired_state_from_position"
    )
    builder.Connect(
        desired_state_from_position.get_output_port(),
        station.GetInputPort("iiwa+suction_base+suction_head.desired_state")
    )

    # Set up differential inverse kinematics.
    diff_ik = AddIiwaDifferentialIK(builder, robot, frame=robot.GetFrameByName("tipLink"))
    builder.Connect(planner.GetOutputPort("X_WG"), diff_ik.get_input_port(0))
    builder.Connect(
        station.GetOutputPort("iiwa+suction_base+suction_head.state_estimated"),
        diff_ik.GetInputPort("robot_state"),
    )
    builder.Connect(
        planner.GetOutputPort("reset_diff_ik"),
        diff_ik.GetInputPort("use_robot_state"),
    )

    # The DiffIK and the direct position-control modes go through a PortSwitch
    switch = builder.AddSystem(PortSwitch(7))
    builder.Connect(
        diff_ik.get_output_port(), switch.DeclareInputPort("diff_ik")
    )
    builder.Connect(
        planner.GetOutputPort("iiwa_position_command"),
        switch.DeclareInputPort("position"),
    )
    builder.Connect(
        switch.get_output_port(), 
        desired_state_from_position.get_input_port()
    )
    builder.Connect(
        planner.GetOutputPort("control_mode"),
        switch.get_port_selector_input_port(),
    )

    scene_graph = station.GetSubsystemByName("scene_graph")
    diagram = builder.Build()

    simulator = Simulator(diagram)
    context = simulator.get_mutable_context()
    station_context = station.GetMyMutableContextFromRoot(context)
    plant_context = station.GetMutableSubsystemContext(plant, station_context)

    X_WS = plant.EvalBodyPoseInWorld(plant_context, plant.GetBodyByName("tipLink"))
    set_board_state(board_state, station, station_context, plant, plant_context, X_WS)
    set_holder_state(holder_state, station, station_context, plant, plant_context, X_WS)
    set_bag_state(bag_state, station, station_context, plant, plant_context, X_WS)

    z = 0.2
    for body_index in plant.GetFloatingBaseBodies():
        tf = RigidTransform(
            UniformlyRandomRotationMatrix(generator),
            [rng.uniform(0.35, 0.65), rng.uniform(-0.12, 0.28), z],
        )
        plant.SetFreeBodyPose(plant_context, plant.get_body(body_index), tf)
        z += 0.1

    # FROM SUCTION CODE 
    scene_graph_context = station.GetMutableSubsystemContext(scene_graph, station_context)
    planner_context = planner.GetMyMutableContextFromRoot(context)
    
    tip_geom_id = scene_graph.model_inspector().GetGeometries(
        plant.GetBodyFrameIdOrThrow(plant.GetBodyByName("tipLink").index()),
        Role.kProximity,
    )[0]

    suction_max_distance = 0.01
    last_suction_state = False
    active_suction_joints = []

    meshcat.Flush()  # Wait for the large object meshes to get to meshcat.
    
    meshcat.StartRecording()
    
    if running_as_notebook:  # Then we're not just running as a test on CI.
        # simulator.set_target_realtime_rate(1)
        meshcat.AddButton("Stop Simulation", "Escape")
        print("Press Escape to stop the simulation")
        while meshcat.GetButtonClicks("Stop Simulation") < 1:
            suction_state = planner.GetOutputPort("suction_on").Eval(
                    planner_context
                )
            print(f"suction state = {suction_state}")
            if suction_state and not last_suction_state:
                print("turning suction on")
                query_object = scene_graph.get_query_output_port().Eval(
                    scene_graph_context
                )
                inspector = query_object.inspector()
                for (pair) in query_object.ComputeSignedDistancePairwiseClosestPoints(suction_max_distance):
                    # Do a little gymnastics to figure out which joint to lock
                    suctioned_geom_id = None
                    if pair.id_A == tip_geom_id:
                        suctioned_geom_id = pair.id_B
                    elif pair.id_B == tip_geom_id:
                        suctioned_geom_id.geom_id = pair.id_A
                    else:
                        continue
                    suctioned_frame_id = inspector.GetFrameId(
                        suctioned_geom_id
                    )
                    suctioned_body = plant.GetBodyFromFrameId(
                        suctioned_frame_id
                    )

                    for i in range(plant.num_joints()):
                        joint = plant.get_joint(JointIndex(i))
                        if joint.child_body() == suctioned_body:
                            joint.Lock(plant_context)
                            active_suction_joints.append(joint)
                            print(f"suctioned the {suctioned_body.name()}")
                if not active_suction_joints:
                    print("too far away to suction anything")

            elif not suction_state and last_suction_state:
                print("turning suction off")
                for joint in active_suction_joints:
                    joint.Unlock(plant_context)
                active_suction_joints = []
            
            last_suction_state = suction_state
            simulator.AdvanceTo(context.get_time() + 0.1)

        meshcat.DeleteButton("Stop Simulation")
    else:
        simulator.set_target_realtime_rate(0)
        simulator.AdvanceTo(context.get_time() + 2.0)

    meshcat.PublishRecording()

run_game()

_______
_______
times: {'initial': 100.4, 'prepare': 101.5380198611403, 'prepick': 102.67603972228059, 'pick_start': 104.67603972228059, 'pick_end': 106.67603972228059, 'postpick': 108.67603972228059, 'clearance': 109.91298151557565, 'preplace': 111.14992330887071, 'place_start': 113.14992330887071, 'place_end': 115.14992330887071, 'postplace': 117.14992330887071}
Game over! Your final score is 4. Please press `esc` to stop the simulation.
suction state = [0.]
mode: PlannerState.END_GAME
current time: 495.3
holder: [None, 'o1', None]
board: 
_______
____a__
____n__
__hat__
_______
_______
_______
times: {'initial': 100.4, 'prepare': 101.5380198611403, 'prepick': 102.67603972228059, 'pick_start': 104.67603972228059, 'pick_end': 106.67603972228059, 'postpick': 108.67603972228059, 'clearance': 109.91298151557565, 'preplace': 111.14992330887071, 'place_start': 113.14992330887071, 'place_end': 115.14992330887071, 'postplace': 117.14992330887071}
Game over! Your final score is 4. Please pres

KeyboardInterrupt: 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=4b17a40c-416d-4160-9e25-b767157b933d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>